In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

# 1. 기본적인 DNN

# 2. 사이킷런(sklearn) 이용
 - 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동. 원핫인코딩은 작동되지 않음.

In [11]:
from sklearn import datasets  # iris 데이터 로드
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [35]:
# 1. 데이터
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print(iris_X[::50], iris_y[::50])
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, 
                                                   test_size=0.1,  # 기본값은 train:test = 0.75:0.25
                                                   random_state=4,
                                                   stratify=iris_y)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)  # 원핫인코딩하기 전 1차원 데이터

[[5.1 3.5 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.3 3.3 6.  2.5]] [0 1 2]
(135, 4) (135,) (15, 4) (15,)


In [13]:
# 사이킷런에서 제공하는 다층퍼셉트론 기반의 분류 알고리즘 api
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), 
                    activation='relu',
                    solver='adam',        # optimizer 설정값
                    max_iter=1000,        # 최대 학습 횟수
                    early_stopping=True,  # 조기종료 활성화
                    n_iter_no_change=10,  # 검증셋 손실함수가 개선되지 않으면 조기종료 (EarlyStopping의 patience)  # 높이면 score가 높아짐
                    validation_fraction=0.1)  # 검증셋 비율

In [14]:
mlp.fit(train_X, train_y)  # 모델

MLPClassifier(early_stopping=True, hidden_layer_sizes=(50, 30), max_iter=1000)

In [15]:
mlp.score(test_X, test_y)

0.9333333333333333

In [19]:
mlp.predict(test_X[0:1])  # 2차원 데이터가 들어가야 함

array([0])

# 3. 클래스를 생성하여 DNN 구현

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [27]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim, ))
#         model.add(Dense(50, activation=activation))
#         model.add(Dense(30, activation=activation))
#         model.add(Dense(output_dim, activation='softmax'))
        model = Sequential([
            Input(input_dim, ),
            Dense(50, activation=activation),
            Dense(30, activation=activation),
            Dense(output_dim, activation='softmax'),
        ])
        # 학습과정 설정
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

In [38]:
# 1. 데이터 생성 및 전처리
# iris_Y = pd.get_dummies(iris_y).to_numpy()
# # print(iris_Y[::50])  # 원핫인코딩
# train_X, test_X, train_Y, test_Y = train_test_split(iris_X, iris_Y,  # 원핫인코딩한 데이터로 분류
#                                                    test_size=0.1,    # 기본값은 train:test = 0.75:0.25
#                                                    random_state=4,
#                                                    stratify=iris_y)
# print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)
train_Y = pd.get_dummies(train_y).values
test_Y = pd.get_dummies(test_y).values

# 2. 모델 생성
# model = DNNClassifier.build()  # 클래스를 생성하여 기본값을 쓰지 않아도 됨
model = DNNClassifier.build(input_dim=4, activation='relu', output_dim=3)

# 3. 모델 학습
earlystopping = EarlyStopping(patience=150)
hist = model.fit(train_X, train_Y, epochs=1000, validation_split=0.1, callbacks=[earlystopping])

Epoch 1/1000
4/4 [==============================] - 1s 72ms/step - loss: 1.1013 - accuracy: 0.3554 - val_loss: 1.3512 - val_accuracy: 0.1429
Epoch 2/1000
4/4 [==============================] - 0s 11ms/step - loss: 1.0596 - accuracy: 0.3554 - val_loss: 1.2759 - val_accuracy: 0.1429
Epoch 3/1000
4/4 [==============================] - 0s 11ms/step - loss: 1.0261 - accuracy: 0.3554 - val_loss: 1.2134 - val_accuracy: 0.1429
Epoch 4/1000
4/4 [==============================] - 0s 12ms/step - loss: 1.0003 - accuracy: 0.3554 - val_loss: 1.1577 - val_accuracy: 0.1429
Epoch 5/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.9746 - accuracy: 0.3554 - val_loss: 1.1099 - val_accuracy: 0.1429
Epoch 6/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.9483 - accuracy: 0.3554 - val_loss: 1.0682 - val_accuracy: 0.1429
Epoch 7/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.9209 - accuracy: 0.3884 - val_loss: 1.0324 - val_accuracy: 0.2857
Epoch 8/1000


Epoch 59/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.2404 - accuracy: 0.9752 - val_loss: 0.2287 - val_accuracy: 0.9286
Epoch 60/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.2346 - accuracy: 0.9752 - val_loss: 0.2226 - val_accuracy: 0.9286
Epoch 61/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.2280 - accuracy: 0.9752 - val_loss: 0.2204 - val_accuracy: 0.9286
Epoch 62/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2239 - accuracy: 0.9752 - val_loss: 0.2224 - val_accuracy: 0.9286
Epoch 63/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.2171 - accuracy: 0.9752 - val_loss: 0.2169 - val_accuracy: 0.9286
Epoch 64/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.2123 - accuracy: 0.9752 - val_loss: 0.2130 - val_accuracy: 0.9286
Epoch 65/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.2097 - accuracy: 0.9835 - val_loss: 0.2072 - val_accuracy: 0.9286
Epoch 

4/4 [==============================] - 0s 11ms/step - loss: 0.0926 - accuracy: 0.9835 - val_loss: 0.1555 - val_accuracy: 0.9286
Epoch 117/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0911 - accuracy: 0.9835 - val_loss: 0.1601 - val_accuracy: 0.9286
Epoch 118/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.0907 - accuracy: 0.9835 - val_loss: 0.1642 - val_accuracy: 0.9286
Epoch 119/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0906 - accuracy: 0.9835 - val_loss: 0.1567 - val_accuracy: 0.9286
Epoch 120/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0885 - accuracy: 0.9835 - val_loss: 0.1572 - val_accuracy: 0.9286
Epoch 121/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0882 - accuracy: 0.9835 - val_loss: 0.1576 - val_accuracy: 0.9286
Epoch 122/1000
4/4 [==============================] - 0s 9ms/step - loss: 0.0886 - accuracy: 0.9835 - val_loss: 0.1627 - val_accuracy: 0.9286
Epoch 123/1000


4/4 [==============================] - 0s 12ms/step - loss: 0.0656 - accuracy: 0.9917 - val_loss: 0.1494 - val_accuracy: 0.9286
Epoch 174/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0656 - accuracy: 0.9917 - val_loss: 0.1466 - val_accuracy: 0.8571
Epoch 175/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0663 - accuracy: 0.9917 - val_loss: 0.1515 - val_accuracy: 0.9286
Epoch 176/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0666 - accuracy: 0.9917 - val_loss: 0.1547 - val_accuracy: 0.9286
Epoch 177/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0647 - accuracy: 0.9835 - val_loss: 0.1506 - val_accuracy: 0.9286
Epoch 178/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0666 - accuracy: 0.9917 - val_loss: 0.1463 - val_accuracy: 0.8571
Epoch 179/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0656 - accuracy: 0.9917 - val_loss: 0.1517 - val_accuracy: 0.9286
Epoch 180/1000

4/4 [==============================] - 0s 11ms/step - loss: 0.0560 - accuracy: 0.9917 - val_loss: 0.1511 - val_accuracy: 0.9286
Epoch 231/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0587 - accuracy: 0.9917 - val_loss: 0.1481 - val_accuracy: 0.8571
Epoch 232/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0550 - accuracy: 0.9917 - val_loss: 0.1524 - val_accuracy: 0.9286
Epoch 233/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0558 - accuracy: 0.9917 - val_loss: 0.1529 - val_accuracy: 0.9286
Epoch 234/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0560 - accuracy: 0.9917 - val_loss: 0.1515 - val_accuracy: 0.9286
Epoch 235/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0565 - accuracy: 0.9917 - val_loss: 0.1518 - val_accuracy: 0.9286
Epoch 236/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.0556 - accuracy: 0.9917 - val_loss: 0.1507 - val_accuracy: 0.9286
Epoch 237/1000

4/4 [==============================] - 0s 12ms/step - loss: 0.0516 - accuracy: 0.9917 - val_loss: 0.1521 - val_accuracy: 0.9286
Epoch 288/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0506 - accuracy: 0.9917 - val_loss: 0.1484 - val_accuracy: 0.8571
Epoch 289/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0514 - accuracy: 0.9917 - val_loss: 0.1471 - val_accuracy: 0.8571
Epoch 290/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0525 - accuracy: 0.9917 - val_loss: 0.1479 - val_accuracy: 0.8571
Epoch 291/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0528 - accuracy: 0.9917 - val_loss: 0.1562 - val_accuracy: 0.9286
Epoch 292/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0513 - accuracy: 0.9917 - val_loss: 0.1519 - val_accuracy: 0.8571
Epoch 293/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0508 - accuracy: 0.9917 - val_loss: 0.1509 - val_accuracy: 0.8571
Epoch 294/1000

4/4 [==============================] - 0s 10ms/step - loss: 0.0501 - accuracy: 0.9835 - val_loss: 0.1510 - val_accuracy: 0.8571
Epoch 345/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.0485 - accuracy: 0.9917 - val_loss: 0.1598 - val_accuracy: 0.9286
Epoch 346/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.0506 - accuracy: 0.9917 - val_loss: 0.1610 - val_accuracy: 0.9286
Epoch 347/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0514 - accuracy: 0.9917 - val_loss: 0.1498 - val_accuracy: 0.8571
Epoch 348/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0490 - accuracy: 0.9917 - val_loss: 0.1503 - val_accuracy: 0.8571
Epoch 349/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0486 - accuracy: 0.9917 - val_loss: 0.1533 - val_accuracy: 0.8571
Epoch 350/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0476 - accuracy: 0.9917 - val_loss: 0.1525 - val_accuracy: 0.8571
Epoch 351/1000

In [45]:
loss, accuracy = model.evaluate(test_X, test_Y)
print(f'정확도 : {accuracy*100}%')
print(f'loss : {loss}')

1/1 [==============================] - 0s 26ms/step - loss: 0.0346 - accuracy: 1.0000
정확도 : 100.0%
loss : 0.0345609188079834


In [48]:
model.predict(test_X[0].reshape(1, -1)).argmax(axis=1)

1/1 [==============================] - 0s 24ms/step


array([0], dtype=int64)

# 4. 함수 api를 이용한 병렬처리

In [52]:
from tensorflow.keras.layers import concatenate  # 병렬 작업 후 출력(Dense들을 연결함)
from tensorflow.keras import Model

In [53]:
train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

((135, 4), (135, 3), (15, 4), (15, 3))

In [55]:
# 병렬처리 할 layer 모델 구성
input_ = Input(shape=(4, ))  # inputlayer
dense1 = Dense(units=50, activation='relu')(input_)  # 4*50+50
dense2 = Dense(units=80, activation='relu')(input_)  # 4*80+80
dense3 = Dense(units=30, activation='relu')(input_)  # 4*30+30
x = concatenate([dense1, dense2, dense3])            # 160
# x = Dense(units=160, activation='relu')(input_)  # 위의 x와 같음. 병렬처리 구현
output = Dense(units=3, activation='softmax')(x)     # 160*3+3
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_37 (Dense)               (None, 50)           250         ['input_14[0][0]']               
                                                                                                  
 dense_38 (Dense)               (None, 80)           400         ['input_14[0][0]']               
                                                                                                  
 dense_39 (Dense)               (None, 30)           150         ['input_14[0][0]']               
                                                                                            

In [57]:
# 학습 정의(학습 설정), 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y, epochs=200, verbose=1)

Epoch 1/200
5/5 [==============================] - 1s 0s/step - loss: 1.2915 - accuracy: 0.3333
Epoch 2/200
5/5 [==============================] - 0s 2ms/step - loss: 1.0573 - accuracy: 0.3333
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 0.9680 - accuracy: 0.6667
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 0.8544 - accuracy: 0.6296
Epoch 5/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7740 - accuracy: 0.6370
Epoch 6/200
5/5 [==============================] - 0s 3ms/step - loss: 0.7168 - accuracy: 0.6667
Epoch 7/200
5/5 [==============================] - 0s 3ms/step - loss: 0.6731 - accuracy: 0.8074
Epoch 8/200
5/5 [==============================] - 0s 0s/step - loss: 0.6362 - accuracy: 0.8370
Epoch 9/200
5/5 [==============================] - 0s 895us/step - loss: 0.6024 - accuracy: 0.6889
Epoch 10/200
5/5 [==============================] - 0s 821us/step - loss: 0.5774 - accuracy: 0.6667
Epoch 11/200
5/5 [=========

5/5 [==============================] - 0s 4ms/step - loss: 0.1664 - accuracy: 0.9852
Epoch 85/200
5/5 [==============================] - 0s 0s/step - loss: 0.1636 - accuracy: 0.9852
Epoch 86/200
5/5 [==============================] - 0s 0s/step - loss: 0.1638 - accuracy: 0.9778
Epoch 87/200
5/5 [==============================] - 0s 5ms/step - loss: 0.1629 - accuracy: 0.9778
Epoch 88/200
5/5 [==============================] - 0s 0s/step - loss: 0.1607 - accuracy: 0.9778
Epoch 89/200
5/5 [==============================] - 0s 4ms/step - loss: 0.1575 - accuracy: 0.9852
Epoch 90/200
5/5 [==============================] - 0s 5ms/step - loss: 0.1557 - accuracy: 0.9852
Epoch 91/200
5/5 [==============================] - 0s 2ms/step - loss: 0.1545 - accuracy: 0.9778
Epoch 92/200
5/5 [==============================] - 0s 8ms/step - loss: 0.1539 - accuracy: 0.9704
Epoch 93/200
5/5 [==============================] - 0s 4ms/step - loss: 0.1513 - accuracy: 0.9778
Epoch 94/200
5/5 [==================

5/5 [==============================] - 0s 3ms/step - loss: 0.0959 - accuracy: 0.9852
Epoch 167/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0948 - accuracy: 0.9778
Epoch 168/200
5/5 [==============================] - 0s 598us/step - loss: 0.0985 - accuracy: 0.9778
Epoch 169/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0955 - accuracy: 0.9704
Epoch 170/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0943 - accuracy: 0.9778
Epoch 171/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0934 - accuracy: 0.9778
Epoch 172/200
5/5 [==============================] - 0s 4ms/step - loss: 0.0913 - accuracy: 0.9852
Epoch 173/200
5/5 [==============================] - 0s 0s/step - loss: 0.0939 - accuracy: 0.9778
Epoch 174/200
5/5 [==============================] - 0s 0s/step - loss: 0.0932 - accuracy: 0.9852
Epoch 175/200
5/5 [==============================] - 0s 0s/step - loss: 0.0919 - accuracy: 0.9778
Epoch 176/200
5/5 [======

In [58]:
# 학습평가
loss, accuracy = model.evaluate(test_X, test_Y)
print(accuracy*100, '%')

1/1 [==============================] - 0s 162ms/step - loss: 0.1174 - accuracy: 0.9333
93.33333373069763 %
